In [1]:
import os
import torch
os.environ['CUDA_VISIBLE_DEVICES'] = "3"
device = torch.device("cuda")

In [2]:
import sys
sys.path.append('./utils/')
sys.path.append('./huggingface_models/')
from leetcode_exec_utils import *
from test_generation_utils import *
from sample_utils import *
from inference_utils import *

In [3]:
%load_ext autoreload
%autoreload 2

### Load all the preprocessed data

In [4]:
import json

# Load the good leetcode data
code_lang_dict = read_leetcode_code_dict(leetcode_code_dict_path)
code_id_lang_dict = get_id_lang_dic(code_lang_dict)

# Generated call code with testcases on all three languages
with open(cached_path + 'leetcode_call_dict.json') as infile:
    call_dict = json.load(infile)
# results on the original/built-in testcases
with open(cached_path + 'leetcode_results_processed.json') as infile:
    results_processed = json.load(infile)
# filtered code_id_lang_dict, only keep the ones that pass all the built-in testcases
with open(cached_path + 'leetcode_filtered_code_id_lang_dict.json') as infile:
    filtered_code_id_lang_dict = json.load(infile)
# generated testcases with diveristy constraints on Java and C++
with open(cached_path + 'leetcode_generated_testcases.json') as infile:
    testcase_id_lang_dict = json.load(infile)
    
# To check tokenization safety (on execution)
leetcode_pids_dict, leetcode_functions_dict, leetcode_functions_toked_dict, \
  leetcode_functions_detoked_dict = get_all_functions_detok_from_cache(
            filtered_code_id_lang_dict, leetcode_pids_dict_path, leetcode_functions_dict_path, 
            leetcode_functions_toked_dict_path, leetcode_functions_detoked_dict_path)
# processed functions_dict
prepro_functions_dict = data_prepro_notok(leetcode_functions_dict, leetcode_functions_toked_dict)

In [53]:
leetcode_pids_dict['Java'][0]

'1700'

In [35]:
imports = []
for pid in code_id_lang_dict['Java'].keys():
    pieces = code_id_lang_dict['Java'][pid]['program_pieces']
    import_lines = pieces[0].split('\n')
    for line in import_lines:
        if line.startswith('import'):
            imports.append(line)
import_set = set(imports)
print(len(import_set))

In [48]:
imports_java = \
[
# 'import com.fishercoder.common.classes.Employee;',
#  'import com.fishercoder.common.classes.ListNode;',
#  'import com.fishercoder.common.classes.NestedInteger;',
#  'import com.fishercoder.common.classes.Node;',
#  'import com.fishercoder.common.classes.Point;',
#  'import com.fishercoder.common.classes.TreeLinkNode;',
#  'import com.fishercoder.common.classes.TreeNode;',
#  'import com.fishercoder.common.utils.CommonUtils;',
#  'import com.fishercoder.common.utils.TreeUtils;',
 'import java.math.BigInteger;',
 'import java.sql.Struct;',
 'import java.util.*;',
 'import static java.lang.Math.max;',
 'import static java.lang.Math.min;']

In [10]:
leetcode_split_dict = get_leetcode_split(None)

In [17]:
lang_pairs = []
for lang1 in tri_langs:
    for lang2 in tri_langs:
        if lang2 != lang1:
            lang_pairs.append((lang1, lang2))

In [5]:
data_name = 'leetcode'
function_data_path = leetcode_pair_data_path + "pair_data_notok_exec_leetcode/"
data_code_dict_path = leetcode_code_dict_path

In [230]:
sample_size = 5
temperature=0.5
tag = "test"
model_type = "plbart"
exp_suffix = "_translation_exec_function/" 
#_translation_transfer_leetcode_exec_function _translation_leetcode_exec_function _translation_exec_function

In [203]:
lang1, lang2 = "Python", "C++"

In [ ]:
# TODO. I created leetcode dataset a bit differently from I thought. 
# I thought I used the whole program, but I actually used just the functions
# does these functions even execute with the generated callcode?

In [62]:
def get_paired_functions_dict(leetcode_split_dict, leetcode_pids_dict, prepro_functions_dict):
    key_set = set()
    for tag in tags:
        key_set = key_set | set(leetcode_split_dict[tag])
    paired_pids_dict = {x:[] for x in tri_langs}
    paired_functions_dict = {}
    for lang, pids in leetcode_pids_dict.items():
        paired_functions_dict[lang] = []
        for i, pid in enumerate(pids):
            if pid in key_set:
                paired_pids_dict[lang].append(pid)
                paired_functions_dict[lang].append(prepro_functions_dict[lang][i])
        print(lang, len(paired_pids_dict[lang]))
    return paired_pids_dict, paired_functions_dict

In [56]:
paired_pids_dict, paired_prepro_functions_dict = get_paired_functions_dict(leetcode_split_dict, 
                                                                    leetcode_pids_dict, prepro_functions_dict)

Python 463
Java 463
C++ 463


In [65]:
single_pids_dict, single_prepro_functions_dict = get_single_functions_dict(leetcode_split_dict, 
                                                                    leetcode_pids_dict, prepro_functions_dict)

Python 539
Java 234
C++ 749


In [66]:
pids = single_pids_dict[lang1]
src_codes = single_prepro_functions_dict[lang1]
tgt_codes = []
is_eval = False

In [231]:
pids = paired_pids_dict[lang2]
src_codes = paired_prepro_functions_dict[lang1]
tgt_codes = paired_prepro_functions_dict[lang2]
is_eval = True

In [232]:
# infer with src and tgt
eval_examples, eval_features, eval_dataloader, model, tokenizer, args, decoder_sid = inference_prepro(
 lang1, lang2, model_type, device, src_codes, tgt_codes, None, tag, exp_suffix)

In [ ]:
# infer with target not from file
eval_examples, eval_features, eval_dataloader, model, tokenizer, args, decoder_sid = inference_prepro(
 lang1, lang2, model_type, device, src_codes, tgt_codes, None, tag, exp_suffix)

In [233]:
preds, eval_result = generation_multiple(eval_examples, eval_dataloader, 
                                             model, tokenizer, args, device, 
                                             decoder_sid, is_eval)

100%|██████████| 463/463 [00:11<00:00, 41.93it/s]


  bleu-4 = 49.7603 
  xMatch = 0.0 
  ********************


In [ ]:
lang = "Java"
code_dic = filtered_code_id_lang_dict['Java']['100']
program = code_dic['program_formatted']
cases = code_dic['test_cases']['tests']
pid = code_dic['idx']
target_info = get_target_info(code_dic, lang)
for j, case in enumerate(cases):
    codestring = generate_call_code(program, case, lang, target_info)
    print(codestring)
    break

In [217]:
def get_hypo_target_info(code_string, target_func, lang):
    code1 =  get_code_for_parsing(lang, code_string)
    hypo_target_func = ""
    target_param = None
    target_return_type = None
    if lang == "Python":
        fns = get_func_names_python(code_string)
    else:
        fns, params, return_types = extract_function_info(code1, lang)
    if len(fns) == 1:
        if lang == "Python":
            hypo_target_func = fns[0]
        else:
            hypo_target_func, target_param, target_return_type = fns[0], params[0], return_types[0]
    elif len(fns) > 1:
        max_sm = 0
        max_sm_id = 0
        for i, fn in enumerate(fns):
            if fn.lower().strip() == target_func.lower().strip():
                max_sm_id = i
                break
            sm = difflib.SequenceMatcher(None, target_func, fn).ratio()
            if sm > max_sm:
                max_sm = sm
                max_sm_id = i
        if lang == "Python":
            hypo_target_func = fns[max_sm_id]
        else:
            hypo_target_func, target_param, target_return_type \
                = fns[max_sm_id], params[max_sm_id], return_types[max_sm_id]
    return hypo_target_func, target_param, target_return_type

# need to convert return type from source to target
def get_hypo_callcode_no_output_leetcode(samples, lang, target_func, cases, is_plbart=False):
    codestring_dict = {}
    codestrings = []
    for i, sample in enumerate(samples):
        sample = notok_detok(sample, lang, is_plbart)
        hypo_target_info = get_hypo_target_info(sample, target_func, lang)
        hypo_target_func = hypo_target_info[0]
        if hypo_target_func == "":
            continue
        target_hypo = sample.replace(hypo_target_func, target_func)
        
        if lang == "Java":
            target_hypo = detok_format(target_hypo, file_detokenizers[lang])
            import_str = "\n".join(imports_java) + '\n'
            target_hypo = import_str + "class GFG {\n" + target_hypo + "\n}"
            hypo_target_func, target_param, target_return_type = hypo_target_info
            para_list = get_para_list(target_param)
            if para_list == None:
                continue
            target_info = ["", target_return_type, target_func, para_list]
        codestring_list = []
        for j, case in enumerate(cases):
            codestring = generate_call_code(target_hypo, case, lang, target_info)
            if codestring == None:
                continue
            codestring_list.append(codestring)
            codestrings += codestring_list
        codestring_dict[i] = codestring_list
    return codestring_dict, codestrings

def prep_exec_hypo_no_output_leetcode(preds, pids, code_id_lang_dict, lang1, lang2, model_type):
    programs = []
    # used to collect all the results for one pid
    program_id_dict = {}
    program_dict = {}
    is_plbart = False
    if model_type == "plbart":
        is_plbart = True
    for i, samples in enumerate(tqdm(preds)):
        pid = pids[i]
        code_dic = code_id_lang_dict[lang1][pid]
        target_info = get_target_info(code_dic, lang1)
        if target_info == None:
            continue
        if lang1 == "Python":
            target_func = target_info[1]
        else:
            target_func = target_info[2]
        cases = code_dic['test_cases']['tests']
        codestring_dict, codestrings = get_hypo_callcode_no_output_leetcode(samples, lang2, target_func,
                                                                            cases, is_plbart)
        program_dict[pid] = codestring_dict
        program_id_dict[pid] = [ind for ind in range(len(programs), len(programs)+len(codestrings))]
        programs += codestrings
    return programs, program_id_dict, program_dict

def get_hypo_callcode_leetcode(samples, lang, code_dic, is_plbart=False):
    target_info = get_target_info(code_dic, lang)
    if target_info == None:
        return None, None
    if lang == "Python":
        target_func = target_info[1]
    else:
        target_func = target_info[2]
    cases = code_dic['test_cases']['tests']
    codestring_dict = {}
    codestrings = []
    for i, sample in enumerate(samples):
        sample = notok_detok(sample, lang, is_plbart)
        hypo_target_func = get_hypo_target_fn(sample, target_func, lang)
        if hypo_target_func == "":
            continue
        target_hypo = sample.replace(hypo_target_func, target_func)
        if lang == "Java":
            target_hypo = detok_format(target_hypo, file_detokenizers[lang])
        if lang == "Python":
            target_hypo = target_hypo.replace(target_func + "(", 
                                              target_func + "(self,").replace(target_func + " (", 
                                                                              target_func + "(self,")
            lines = target_hypo.split('\n')
            lines = ["    " + x for x in lines]
            target_hypo = "\n".join(lines)
        pieces = code_dic['program_pieces']
        pieces[0] = pieces[0].strip() + "\n"
        if target_function_place_holder not in pieces:
            pieces = pieces[:-1] + [target_function_place_holder] + [pieces[-1]]
#         print("".join(pieces))
        program = "".join(pieces).replace(target_function_place_holder, target_hypo)
        codestring_list = []
        for j, case in enumerate(cases):
            codestring = generate_call_code(program, case, lang, target_info)
            if codestring == None:
                continue
            codestring_list.append(codestring)
            codestrings += codestring_list
        codestring_dict[i] = codestring_list
    return codestring_dict, codestrings

def prep_exec_hypo_leetcode(preds, pids, code_id_lang_dict, lang, model_type):
    programs = []
    program_id_dict = {}
    program_dict = {}
    is_plbart = False
    if model_type == "plbart":
        is_plbart = True
    for i, samples in enumerate(tqdm(preds)):
        pid = pids[i]
        code_dic = code_id_lang_dict[lang][pid]
        codestring_dict, codestrings = get_hypo_callcode_leetcode(samples, lang, code_dic, is_plbart)
        if codestring_dict == None:
            continue
        program_dict[pid] = codestring_dict
        program_id_dict[pid] = [ind for ind in range(len(programs), len(programs)+len(codestrings))]
        programs += codestrings
    return programs, program_id_dict, program_dict

def prep_exec_programs_leetcode(prepro_funcs, pids, code_id_lang_dict, lang, model_type):
    programs = []
    program_id_dict = {}
    program_dict = {}
    is_plbart = False
    if model_type == "plbart":
        is_plbart = True
    
    for i, prepro_func in enumerate(tqdm(prepro_funcs)):
        pid = pids[i]
        code_dic = code_id_lang_dict[lang][pid]
        target_info = get_target_info(code_dic, lang)
        cases = code_dic['test_cases']['tests']
        prepro_func = notok_detok(prepro_func, lang, is_plbart)
        if lang == "Java":
            prepro_func = detok_format(prepro_func, file_detokenizers[lang])
        if lang == "Python":
            lines = prepro_func.split('\n')
            lines = ["    " + x for x in lines]
            prepro_func = "\n".join(lines)
        pieces = code_dic['program_pieces']
        pieces[0] = pieces[0].strip() + "\n"
        if target_function_place_holder not in pieces:
            pieces = pieces[:-1] + [target_function_place_holder] + [pieces[-1]]
        program = "".join(pieces).replace(target_function_place_holder, prepro_func)
        codestrings = []
        for j, case in enumerate(cases):
            codestring = generate_call_code(program, case, lang, target_info)
            if codestring == None:
                print(pid, j)
                continue
            codestrings.append(codestring)
        program_id_dict[pid] = [ind for ind in range(len(programs), len(programs)+len(codestrings))]
        programs += codestrings
    return programs, program_id_dict

In [154]:
for pid, code in zip(pids, tgt_codes):
    if pid == '680':
        print(code)
        break

def validPalindrome ( self , s ) : NEW_LINE INDENT def validPalindrome ( s , left , right ) : NEW_LINE INDENT while left < right : NEW_LINE INDENT if s [ left ] != s [ right ] : NEW_LINE INDENT return False NEW_LINE DEDENT left , right = left + 1 , right - 1 NEW_LINE DEDENT return True NEW_LINE DEDENT left , right = 0 , len ( s ) - 1 NEW_LINE while left < right : NEW_LINE INDENT if s [ left ] != s [ right ] : NEW_LINE INDENT return validPalindrome ( s , left , right - 1 ) or validPalindrome ( s , left + 1 , right ) NEW_LINE DEDENT left , right = left + 1 , right - 1 NEW_LINE DEDENT return True NEW_LINE DEDENT NEW_LINE def validPalindrome ( s , left , right ) : NEW_LINE INDENT while left < right : NEW_LINE INDENT if s [ left ] != s [ right ] : NEW_LINE INDENT return False NEW_LINE DEDENT left , right = left + 1 , right - 1 NEW_LINE DEDENT return True NEW_LINE DEDENT


In [218]:
# test preds on testcases
# num_datapoints = 100 [:num_datapoints]
programs, program_id_dict = prep_exec_programs_leetcode(tgt_codes, pids, 
                                                filtered_code_id_lang_dict, lang2, model_type)
print(len(programs))
# lang_results = p_map(file_executors[lang], programs)


100%|██████████| 463/463 [00:00<00:00, 16932.72it/s]

1327


In [211]:
lang = lang2
lang_results = p_map(file_executors[lang], programs)


In [212]:
result_type_dict = show_result_summary({lang:lang_results})

result_id_dict, pass_list, fail_list, error_type_dict = get_pass_fail_error(lang_results, 
                                                                        program_id_dict, pids[:num_datapoints], lang)
pass_result_id_dict = {x:result_id_dict[x] for x in pass_list}
error_type_dict

C++ error 141 0.10625470987189148
C++ timeout 0 0.0
C++ empty 0 0.0
C++ other 0 0.0
C++ good 1186 0.8937452901281085


{'N was not declared in': 2,
 'class Solution has no member': 2,
 'int Solution::rob(std::vector<int>&) is private within': 2,
 'op was not declared in': 2,
 'expected unqualified-id before private': 3,
 'int Solution::lengthOfLIS(std::vector<int>&) is private within': 3,
 'split was not declared in': 4,
 'solve was not declared in': 5,
 'data was not declared in': 5,
 'expected declaration before } token': 29}

In [234]:
# test preds on testcases
num_datapoints = 100
programs, program_id_dict, program_dict = prep_exec_hypo_leetcode(preds, pids, 
                                                filtered_code_id_lang_dict, lang2, model_type)
call_dict[(lang1, lang2)] = pids, programs, program_id_dict, program_dict
print(len(programs))
# lang_results = p_map(file_executors[lang], programs)


  5%|▍         | 21/463 [00:00<00:04, 99.01it/s]

#include <iostream>
#include <vector>
using namespace std;
class Solution {
public:
	vector<int> twoSum(vector<int>& nums, int target) {
		for(int i = 0 ; i < nums.size() ; i ++)
			for(int j = i + 1 ; j < nums.size() ; j ++)
				if(nums[i] + nums[j] == target)
					return {i, j};
		throw invalid_argument("the input has no solution");
	}
};

<target_function_place_holder>

};
void print_vec(const vector<int>& vec){
	for(int e: vec)
		cout << e << " ";
	cout << endl;
}
int main() {
	vector<int> nums = {0,4,3,5};
	int target = 10;
	print_vec(Solution().twoSum(nums, target));
	return 0;
}
#include <iostream>
#include <vector>
using namespace std;
class Solution {
public:
	vector<int> twoSum(vector<int>& nums, int target) {
		for(int i = 0 ; i < nums.size() ; i ++)
			for(int j = i + 1 ; j < nums.size() ; j ++)
				if(nums[i] + nums[j] == target)
					return {i, j};
		throw invalid_argument("the input has no solution");
	}
};

<target_function_place_holder>

};
void print_vec(const vector<

  7%|▋         | 33/463 [00:00<00:04, 102.49it/s]

#include <iostream>
#include <vector>
using namespace std;
class Solution {
private:
	int d[4][2] = {{0, 1}, {1, 0}, {0, -1}, {-1, 0}};
	int N, M;
public:
	vector<int> spiralOrder(vector<vector<int>>& matrix) {
		N = matrix.size();
		if(N == 0)
			return {};
		M = matrix[0].size();
		if(M == 0)
			return {};
		vector<vector<bool>> visited(N, vector<bool>(M, false));
		int curd = 0, curx = 0, cury = 0;
		vector<int> res;
		while(res.size() < N * M){
			if(!visited[curx][cury]) {
				res.push_back(matrix[curx][cury]);
				visited[curx][cury] = true;
			}
			int nextx = curx + d[curd][0];
			int nexty = cury + d[curd][1];
			if(inArea(nextx, nexty) && !visited[nextx][nexty]){
				curx = nextx;
				cury = nexty;
			}
			else
				curd = (curd + 1) % 4;
		}
		return res;
	}
private:
	bool inArea(int x, int y){
		return x >= 0 && x < N && y >= 0 && y < M;
	}
};

<target_function_place_holder>

private:
	
};
void print_vec(const vector<int>& vec){
	for(int e: vec)
		cout << e << " ";
	cout << 

 10%|█         | 47/463 [00:00<00:03, 108.74it/s]

...

#include <iostream>
#include <vector>
using namespace std;
class Solution {
public:

<target_function_place_holder>

};
int main() {
	vector<int> baseCosts1 = {1, 7}, toppingCosts1 = {3, 4};
	cout << Solution().closestCost(baseCosts1, toppingCosts1, 10) << endl;
	vector<int> baseCosts2 = {2, 3}, toppingCosts2 = {4, 5, 100};
	cout << Solution().closestCost(baseCosts2, toppingCosts2, 18) << endl;
	vector<int> baseCosts3 = {3, 10}, toppingCosts3 = {2, 5};
	cout << Solution().closestCost(baseCosts3, toppingCosts3, 9) << endl;
	vector<int> baseCosts4 = {10}, toppingCosts4 = {1};
	cout << Solution().closestCost(baseCosts4, toppingCosts4, 1) << endl;
	vector<int> baseCosts5 = {5, 9}, toppingCosts5 = {10, 1};
	cout << Solution().closestCost(baseCosts5, toppingCosts5, 7) << endl;
	return 0;
}
#include <iostream>
#include <vector>
#include <numeric>
using namespace std;
class Solution {
public:

<target_function_place_holder>

};
int main() {
	vector<int> nums11 = {1,2,3,4,5,6}, nums12 = {1,1,2,

 99%|█████████▉| 460/463 [00:04<00:00, 96.30it/s] 

#include <iostream>
#include <vector>
#include <unordered_map>
#include <unordered_set>
using namespace std;
class Solution {
public:

<target_function_place_holder>

};
int main() {
	return 0;
}
#include <iostream>
#include <vector>
#include <unordered_map>
#include <unordered_set>
using namespace std;
class Solution {
public:

<target_function_place_holder>

};
int main() {
	return 0;
}
#include <iostream>
#include <vector>
#include <unordered_map>
#include <unordered_set>
using namespace std;
class Solution {
public:

<target_function_place_holder>

};
int main() {
	return 0;
}
#include <iostream>
#include <vector>
#include <unordered_map>
#include <unordered_set>
using namespace std;
class Solution {
public:

<target_function_place_holder>

};
int main() {
	return 0;
}
#include <iostream>
#include <vector>
#include <unordered_map>
#include <unordered_set>
using namespace std;
class Solution {
public:

<target_function_place_holder>

};
int main() {
	return 0;
}
#include <iostream>


100%|██████████| 463/463 [00:04<00:00, 94.62it/s]

#include <iostream>
#include <vector>
using namespace std;
class Solution {
public:

<target_function_place_holder>

private:
	
};
int main() {
	cout << Solution().numberOfRounds("12:01", "12:44") << endl;
	cout << Solution().numberOfRounds("20:00", "06:00") << endl;
	cout << Solution().numberOfRounds("00:00", "23:59") << endl;
	cout << Solution().numberOfRounds("23:95", "00:00") << endl;
	return 0;
}
#include <iostream>
#include <vector>
using namespace std;
class Solution {
public:

<target_function_place_holder>

private:
	
};
int main() {
	cout << Solution().numberOfRounds("12:01", "12:44") << endl;
	cout << Solution().numberOfRounds("20:00", "06:00") << endl;
	cout << Solution().numberOfRounds("00:00", "23:59") << endl;
	cout << Solution().numberOfRounds("23:95", "00:00") << endl;
	return 0;
}
#include <iostream>
#include <vector>
using namespace std;
class Solution {
public:

<target_function_place_holder>

private:
	
};
int main() {
	cout << Solution().numberOfRounds("12:01", "1

In [235]:
lang_results = p_map(file_executors[lang2], programs)

In [236]:
# TODO. Update the pass/fail count by samples
# result_type_dict = show_result_summary(results_dict)
lang = lang2
result_type_dict = show_result_summary({lang:lang_results})
pids, programs, program_id_dict, program_dict = call_dict[(lang1, lang2)]
result_id_dict, pass_list, fail_list, error_type_dict = get_pass_fail_error(lang_results, 
                                                                        program_id_dict, pids, lang)
pass_result_id_dict = {x:result_id_dict[x] for x in pass_list}
results_processed[lang] = (pass_result_id_dict, result_id_dict, pass_list, fail_list, error_type_dict)
print('-'*10)
print(lang, "exec_rate:",len(pass_list)/len(pids), len(pass_list), len(fail_list))
print(error_type_dict)
print('*'*10)

C++ error 13113 0.9855693348365276
C++ timeout 0 0.0
C++ empty 5 0.0003757985719654265
C++ other 0 0.0
C++ good 187 0.014054866591506952
----------
C++ exec_rate: 0.0064794816414686825 3 460
{'expected unqualified-id before private': 233, 'a function-definition is not allowed': 251, 'sum was not declared in': 275, 'set was not declared in': 287, 'expected } at end of': 467, 'unordered_map was not declared in': 504, 'request for member size in': 636, 'expected ; at end of': 764, 'map was not declared in': 830, 'no matching function for call': 2024}
**********


In [ ]:
for i, idx in enumerate(fail_list):
    for j, error in enumerate(result_id_dict[idx]):
        if "no matching function for call" in error:
            print(j, idx, error)
            break

In [32]:
lang = "Java"
code_dic = filtered_code_id_lang_dict['Java']['100']
program = code_dic['program_formatted']
cases = code_dic['test_cases']['tests']
pid = code_dic['idx']
target_info = get_target_info(code_dic, lang)
print(target_info)
# for j, case in enumerate(cases):
#     codestring = generate_call_code(program, case, lang, target_info)
#     print(codestring)
#     break

('Solution1', 'boolean', 'isSameTree', [('p', 'TreeNode'), ('q', 'TreeNode')])


#### How the good leetcode data is created

In [ ]:
# leetcode data first version
code_lang_dict_original = read_updated_code_dict(code_dict_path)
code_id_lang_dict_original = get_id_lang_dic(code_lang_dict_original)

# leetcode data from new paths
# code_lang_dict_new = read_updated_code_dict(code_dict_path_new)
# code_id_lang_dict_new = get_id_lang_dic(code_lang_dict_new)

# leetcode data with cpp from readme (py and java same as above)
# code_lang_dict_readme = read_updated_code_dict(code_dict_path_readme)
# code_id_lang_dict_readme = get_id_lang_dic(code_lang_dict_readme)

# leetcode data with cpp from liuyubo repo
code_lang_dict_liuyubo = read_updated_code_dict(code_dict_path_liuyubo)
code_id_lang_dict_liuyubo = get_id_lang_dic(code_lang_dict_liuyubo)

code_id_lang_dict = code_id_lang_dict_original
# code_id_lang_dict = code_id_lang_dict_new
code_id_lang_dict['C++'] = code_id_lang_dict_liuyubo['C++']

#### Load the good data

In [3]:
code_lang_dict = read_leetcode_code_dict(code_dict_path)
code_id_lang_dict = get_id_lang_dic(code_lang_dict)

### Execute C++ programs with test cases

#### Debug code

In [ ]:
# key = pids[421]
lang = "C++"
key = '2096'
test_case = code_id_lang_dict[lang][key]['test_cases']['tests'][0]
print(test_case)
code_dic = code_id_lang_dict[lang][key]
program, func_name_list = code_dic['program_formatted'], code_dic['function_names']
para_lists, return_type_list = code_dic['parameter_lists'], code_dic['return_types']
print(program)
print(func_name_list)
print(para_lists)
print(return_type_list)
target_info = get_target_info_cpp(code_id_lang_dict[lang][key])
if target_info == None:
    print("No target_info!")
print(target_info)
a = generate_cpp_callcode(program, test_case, target_info)
print(a)

In [ ]:
b = run_exec_cpp(a)
print(b.encode("ascii", "ignore").decode())

#### Generate original programs to run without callcode

In [ ]:
programs, program_id_dict, pids = prepare_exec_liuyubo_cpp_originial(code_id_lang_dict)

#### Generate programs to run

In [6]:
lang = "C++"
programs, program_id_dict, pids = prepare_exec_program(code_id_lang_dict, lang)

In [7]:
len(programs)

4235

#### Parallel execution

In [8]:
cpp_results = p_map(run_exec_cpp, programs)

#### Result analysis

In [19]:
result_dict = {lang:cpp_results}
result_type_dict = show_result_summary(result_dict)
result_id_dict, pass_list, fail_list, error_type_dict = get_pass_fail_error(cpp_results, 
                                                                            program_id_dict, pids, lang)
print(len(pass_list), len(fail_list))
error_type_dict

C++ error 611 0.14427390791027156
C++ timeout 2 0.0004722550177095632
C++ empty 5 0.0011806375442739079
C++ other 4 0.0009445100354191264
C++ good 3613 0.8531286894923259
1268 555


{'template argument 1 is invalid': 12,
 'scalar object root1 requires one': 13,
 'cannot bind non-const lvalue reference': 14,
 'invalid conversion from const char*': 14,
 'scalar object expected requires one': 17,
 'invalid conversion from int to': 27,
 'variable or field execution_output declared': 29,
 'expected primary-expression before ; token': 33,
 'no matching function for call': 93,
 'scalar object root requires one': 221}

In [ ]:
for i, idx in enumerate(fail_list):
    for error in result_id_dict[idx]:
        error = error.encode("ascii", "ignore").decode()[:1000]
        if "scalar" in error:
            print(idx, error)
            print('-'*100)
            break

### Execute Java programs with test cases

#### Debug code

In [ ]:
lang = "Java"
key = '206'
test_case = code_id_lang_dict[lang][key]['test_cases']['tests'][0]
print(test_case)
code_dic = code_id_lang_dict[lang][key]
program, func_name_list = code_dic['program_formatted'], code_dic['function_names']
para_lists, return_type_list = code_dic['parameter_lists'], code_dic['return_types']
# print(program)
print(func_name_list)
print(para_lists)
print(return_type_list)
target_info = get_target_info_java(code_id_lang_dict[lang][key])
if target_info == None:
    print("No target_info!")
print(target_info)
a = generate_java_callcode(program, test_case, target_info)
print(a)

In [ ]:
b = run_exec_java_test(a)
print(b)

#### Generate programs to run

In [ ]:
no_code_list = []
for key, code_dic in code_id_lang_dict[lang].items():
#     print(key)
    codestring_list = []
    bad_testcase_list = []
    program = code_dic['program_formatted']
    cases = code_dic['test_cases']['tests']
    pid = code_dic['idx']
    target_info = get_target_info(code_dic, lang)
    for j, case in enumerate(cases):
        codestring = generate_call_code(program, case, lang, target_info)
        if codestring == None:
            bad_testcase_list.append((pid, j))
            continue
        codestring_list.append(codestring)
    if len(codestring_list) == 0:
        no_code_list.append(pid)

In [ ]:
lang = 'Java'
programs, program_id_dict, pids = prepare_exec_program(code_id_lang_dict, lang)

In [ ]:
len(programs)

#### Parallel execution

In [ ]:
java_results = p_map(run_exec_java, programs)

#### Result analysis

In [ ]:
result_dict = {lang:java_results}
result_type_dict = show_result_summary(result_dict)
result_id_dict, pass_list, fail_list, error_type_dict = get_pass_fail_error(java_results, 
                                                                            program_id_dict, pids, lang)
print(len(pass_list), len(fail_list))
error_type_dict

In [ ]:
for i, idx in enumerate(fail_list):
    for error in result_id_dict[idx]:
        if "no suitable method found for" in error:
            print(idx, error)

### Execute Python programs with test cases

#### Debug code

In [161]:
lang = "Python"
key = '1700'
test_case = code_id_lang_dict[lang][key]['test_cases']['tests'][0]
code_dic = code_id_lang_dict[lang][key]
program, func_name_list = code_dic['program_formatted'], code_dic['function_names']
para_lists, return_type_list = code_dic['parameter_lists'], code_dic['return_types']
target_info = get_target_info_python(code_id_lang_dict[lang][key])
if target_info == None:
    print("No target_info!")
print(target_info)
a = generate_python_callcode(program, test_case, target_info)
print(a)

('Solution', 'countStudents')
import collections
import itertools
import math
import random
import operator

import collections
class Solution(object):
	def countStudents(self, students, sandwiches):
		count = collections.Counter(students)
		for i, s in enumerate(sandwiches):
			if not count[s]:
				break
			count[s] -= 1
		else:
			i = len(sandwiches)
		return len(sandwiches)-i
sol = Solution()
students = [1,1,0,0]
sandwiches = [0,1,0,1]
expected = 0
execution_output = sol.countStudents(students, sandwiches)
print(execution_output)
print('*'*10)
if expected == execution_output:
	print(True)
else:
	print(False)


In [162]:
b = run_exec_python(a)
print(b)

0
**********
True


#### Generate programs to run

In [163]:
lang = "Python"
programs, program_id_dict, pids = prepare_exec_program(code_id_lang_dict, lang)

In [164]:
len(programs)

3851

#### Parallel Execution

In [168]:
py_results = p_map(run_exec_python, programs)

#### Result analysis

In [169]:
result_dict = {lang:py_results}
result_type_dict = show_result_summary(result_dict)
result_id_dict, pass_list, fail_list, error_type_dict = get_pass_fail_error(py_results, 
                                                                            program_id_dict, pids, lang)
print(len(pass_list), len(fail_list))
error_type_dict

Python error 516 0.13399117112438327
Python timeout 0 0.0
Python empty 0 0.0
Python other 0 0.0
Python good 3335 0.8660088288756167
1129 208


{'IndexError': 1,
 'ValueError': 1,
 'ImportError': 2,
 '    if index[0] == len(preorder) or TypeError': 2,
 'IndentationError': 24,
 'AttributeError': 36,
 'NameError': 124,
 'TypeError': 136,
 'SyntaxError': 190}

In [167]:
for i, idx in enumerate(fail_list):
    for error in result_id_dict[idx]:
        if "NameError" in error:
            print(idx, error)

1000 Traceback (most recent call last):
  File "./tmp_execution_results/1660101726420.68290.681829524096162.py", line 25, in <module>
    execution_output = sol.mergeStones(stones, k)
  File "./tmp_execution_results/1660101726420.68290.681829524096162.py", line 14, in mergeStones
    dp = [[0]*len(stones) for _ in xrange(len(stones))]
NameError: name 'xrange' is not defined
1000 Traceback (most recent call last):
  File "./tmp_execution_results/1660101726425.94340.3896188150469331.py", line 25, in <module>
    execution_output = sol.mergeStones(stones, k)
  File "./tmp_execution_results/1660101726425.94340.3896188150469331.py", line 14, in mergeStones
    dp = [[0]*len(stones) for _ in xrange(len(stones))]
NameError: name 'xrange' is not defined
1566 Traceback (most recent call last):
  File "./tmp_execution_results/1660101726426.8050.11947064234615756.py", line 23, in <module>
    execution_output = sol.containsPattern(arr, m, k)
  File "./tmp_execution_results/1660101726426.8050.1194

### Execute Programs with original Testcases

In [ ]:
results_dict = {}
call_dict = {}
for lang in tri_langs:
    print(lang)
    programs, program_id_dict, pids = prepare_exec_program(code_id_lang_dict, lang)
    call_dict[lang] = programs, program_id_dict, pids
    print(len(programs))
    lang_results = p_map(file_executors[lang], programs)
    results_dict[lang] = lang_results

In [ ]:
# result_type_dict = show_result_summary(results_dict)
results_processed = {}
for lang in tri_langs:
    results = results_dict[lang]
    result_type_dict = show_result_summary({lang:results})
    programs, program_id_dict, pids = call_dict[lang]
    result_id_dict, pass_list, fail_list, error_type_dict = get_pass_fail_error(results, 
                                                                            program_id_dict, pids, lang)
    pass_result_id_dict = {x:result_id_dict[x] for x in pass_list}
    results_processed[lang] = (pass_result_id_dict, result_id_dict, pass_list, fail_list, error_type_dict)
    print('-'*10)
    print(lang, "exec_rate:",len(pass_list)/len(pids), len(pass_list), len(fail_list))
    print(error_type_dict)
    print('*'*10)

In [ ]:
pass_pids_dict = get_pass_pids_dict(call_dict, results_processed)
# get programs that pass all the tests
filtered_code_id_lang_dict = get_filtered_code_id_lang_dict(pass_pids_dict, code_id_lang_dict)

### Create pairwise data on programs pass all tests

#### Create data split

In [ ]:
# Split ratio: 0.7 0.1 0.2
leetcode_split_dict = get_leetcode_split(pass_pids_dict)

#### Create pairwise data (without tokenization)

In [5]:
leetcode_pids_dict, leetcode_functions_dict, leetcode_functions_toked_dict, \
  leetcode_functions_detoked_dict = get_all_functions_detok_from_cache(
            filtered_code_id_lang_dict, leetcode_pids_dict_path, leetcode_functions_dict_path, 
            leetcode_functions_toked_dict_path, leetcode_functions_detoked_dict_path)

In [ ]:
prepro_functions_dict = data_prepro_notok(leetcode_functions_dict, leetcode_functions_toked_dict)
get_pair_data_notok(leetcode_pair_data_path, "pair_data_notok_exec_leetcode", 
                    leetcode_pids_dict, prepro_functions_dict, 
                    leetcode_split_dict['test'], leetcode_split_dict['val'])

#### Get sequence length (number of tokens)

In [ ]:
from transformers import AutoTokenizer
baseline_models = ['codebert', 'plbart', 'codet5']
tokenizer_dict = {model_type:AutoTokenizer.from_pretrained(baseline_model_dict[model_type])
                  for model_type in baseline_models}

In [ ]:
length_lang_dict = get_length_lang_dict(prepro_functions_dict, tokenizer_dict['codet5'], False) 

### Generate testcases based on types

#### Get target input types

In [ ]:
pass_type_dict = get_pass_java_types(results_processed, code_id_lang_dict)
pass_types_set = set(list(pass_type_dict.keys()))
all_types = get_all_acceptable_types()
all_types_set = set(all_types)

In [14]:
def test_generate_tests(lang):
#     for i, pid in enumerate(results_processed[lang][0].keys()):
    for i, pid in enumerate(code_id_lang_dict[lang].keys()):
        target_info = get_target_info(code_id_lang_dict[lang][pid], lang)
        if target_info == None:
            continue
        new_para_list = target_info[-1]
        for para_name, para_type in new_para_list:
            if "[]" in para_name:
                print(pid, para_name)
                break
#         print(pid, new_para_list)
#         a = generate_tests(new_para_list, 5, lang)
#         print(a)
#         cases = code_id_lang_dict[lang][pid]['test_cases']['tests']
#         print(cases)
#         print("*"*20)
#         if i > 100:
#             break
    return 
lang = "C++" # Java
test_generate_tests(lang)

#### Automated test case generation with diversity constraint to ensure coverage

In [ ]:
# TODO. C++'s array definition is different. int a[2][5] will be classified as int, not int 2d.

In [ ]:
lang = "C++"
for pid, code_dic in filtered_code_id_lang_dict[lang].items():
    target_info = get_target_info(code_id_lang_dict[lang][pid], lang)
    target_cls_name, return_type, target_func_name, para_list = target_info
    real_type, dim = get_real_type(return_type, lang)
    if dim > 0 and real_type == 'int':
        print(pid)

In [5]:
empty_target_lang_dic = {}
for lang in code_id_lang_dict.keys():
    empty_target_lang_dic[lang] = []
    for pid, code_dic in code_id_lang_dict[lang].items():
        target_call = code_dic['return_types']
        if len(target_call) == 0:
            empty_target_lang_dic[lang].append(pid)
        elif target_call[0] == "":
            empty_target_lang_dic[lang].append(pid)
    print(lang, len(empty_target_lang_dic[lang]))

Python 1337
Java 5
C++ 1


In [22]:
code_dic.keys()

dict_keys(['functions', 'program_pieces', 'function_names', 'parameter_lists', 'return_types', 'target_call', 'target_call_args', 'target_call_params', 'target_call_return_type', 'idx', 'program_formatted', 'test_cases', 'tokens', 'bpe'])

In [9]:
lang = "C++"
num_tests = 5
pid = '321'
code_dic = filtered_code_id_lang_dict[lang][pid]
target_info = get_target_info(code_id_lang_dict[lang][pid], lang)
target_cls_name, return_type, target_func_name, para_list = target_info
cases = code_dic['test_cases']['tests']
# print(cases)
print(return_type)

codestring_list, good_cases = get_call_dict_no_output(code_dic, lang, num_tests)
print(good_cases)

vector<int>
[{'inputs': 'nums1 = [44, 47, 64, 67, 67, 9, 83, 21, 36]\nnums2 = [49, 29, 19, 19, 14]\nk = 28\n'}, {'inputs': 'nums1 = [87, 70, 88]\nnums2 = [39, 32]\nk = 34\n'}, {'inputs': 'nums1 = [88, 12, 58, 65, 39, 87, 46, 88, 81]\nnums2 = [65, 9, 57, 32, 31]\nk = 0\n'}, {'inputs': 'nums1 = [37, 25, 77, 72, 9, 20]\nnums2 = [74]\nk = 0\n'}, {'inputs': 'nums1 = [80, 69, 79, 47, 64, 82, 99, 88]\nnums2 = [23, 35, 75, 55]\nk = 36\n'}]


In [10]:
a = file_executors[lang](codestring_list[0])
print(a.encode("ascii", "ignore").decode())

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


#### Main loop to generate testcases on Java and C++ based on data types

In [ ]:
testcase_id_lang_dict = generate_testcases_with_diversity(leetcode_pids_dict, filtered_code_id_lang_dict, 
                                                          num_tests=10)

#### Generate testcases based on existing testcases

In [ ]:
# train on good code, but can generate on buggy input?

In [ ]:
# two ways for testcase generation:
# generate based on existing testcases. Can start with any lang, including Python
# In python, there are string, int, boolean, float, list, list of list as input types.

In [ ]:
code_id_lang_dict['Java']['237'].keys()